# Demo: Review Analyzer Pipeline

**Purpose:** Demonstrate the end-to-end review analysis pipeline for a presentation.

**What this notebook does:**
1. **Discovery** — Find Google Maps place IDs for businesses in a city
2. **Collection** — Collect reviews from discovered locations
3. **Transform** — Normalize and enrich review data
4. **Classification** — Classify reviews using OpenAI GPT-4
5. **EDA** — Visualize results with charts and heatmaps

**Demo Scenario:**
- Business type: Insurance ("assurance")
- Target business: RMA Assurance
- City: Casablanca
- Competitors: User-defined list

**Limits (for demo speed):**
- Max 5 locations per business
- Max 5 reviews per location

---


## Setup & Imports


In [1]:
# Setup: Add project to path and import modules
import sys
import os
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Project root
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root / "src"))

# Core imports
import pandas as pd
import numpy as np
import json

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from IPython.display import display, HTML

# Pipeline modules
from review_analyzer.discover import DiscoveryEngine
from review_analyzer.collect import ReviewCollector
from review_analyzer.classify import ReviewClassifier
from review_analyzer.transformers.normalize_reviews import normalize_reviews_df
from review_analyzer.transformers.geocode import add_region
from review_analyzer import config

print("All imports successful!")
print(f"Project root: {project_root}")


All imports successful!
Project root: /Users/rahalimyriam/Library/CloudStorage/OneDrive-SharedLibraries-TheBostonConsultingGroup,Inc/Bouayad, Halim - SG GeoAnalytics/6. Working Folders/00. Myriam/Work/C__Reviews_Text/review_analyzer


## Demo Configuration

**Edit the variables below to customize the demo:**

- `BUSINESS_TYPE`: The type of business (e.g., "assurance", "banque", "restaurant")
- `TARGET_BUSINESS`: The main business to analyze
- `COMPETITORS`: List of competitor businesses to compare
- `CITY`: The city to search in


In [11]:
# ==============================================================================
# DEMO CONFIGURATION — EDIT THESE VALUES
# ==============================================================================

# Business type (used for search context)
BUSINESS_TYPE = "assurance"

# Target business name
TARGET_BUSINESS = "RMA Assurance"

# Competitors to compare against (edit this list as needed!)
# Suggested competitors for insurance in Morocco:
COMPETITORS = [
    "Wafa Assurance",
    # "Saham Assurance",
    "Axa Assurance Maroc",
    # "Atlanta Assurance",  # Uncomment to add more
    # "Allianz Maroc",
]

# City to search in
CITY = "Casablanca"

# ==============================================================================
# DEMO LIMITS — Keep these for fast demo execution
# ==============================================================================

MAX_LOCATIONS_PER_BUSINESS = 5  # Max locations to discover per business
MAX_REVIEWS_PER_LOCATION = 5     # Max reviews to collect per location

# ==============================================================================

# Combine target + competitors into full list
ALL_BUSINESSES = [TARGET_BUSINESS] + COMPETITORS

print("DEMO CONFIGURATION")
print("=" * 60)
print(f"   Business Type: {BUSINESS_TYPE}")
print(f"   Target Business: {TARGET_BUSINESS}")
print(f"   Competitors: {COMPETITORS}")
print(f"   City: {CITY}")
print(f"\n   Limits:")
print(f"      Max locations per business: {MAX_LOCATIONS_PER_BUSINESS}")
print(f"      Max reviews per location: {MAX_REVIEWS_PER_LOCATION}")
print(f"\n   Total businesses to analyze: {len(ALL_BUSINESSES)}")
print(f"   Estimated max reviews: {len(ALL_BUSINESSES) * MAX_LOCATIONS_PER_BUSINESS * MAX_REVIEWS_PER_LOCATION}")


DEMO CONFIGURATION
   Business Type: assurance
   Target Business: RMA Assurance
   Competitors: ['Wafa Assurance', 'Axa Assurance Maroc']
   City: Casablanca

   Limits:
      Max locations per business: 5
      Max reviews per location: 5

   Total businesses to analyze: 3
   Estimated max reviews: 75


---

## Step 1: Discovery

Find Google Maps place IDs for all businesses in the specified city.

**What happens:**
- Uses SerpAPI to search Google Maps
- Resolves canonical place IDs (ChIJ... format)
- Returns location details (name, address, coordinates)


In [12]:
# ==============================================================================
# STEP 1: DISCOVERY — Find Place IDs
# ==============================================================================

print("STEP 1: DISCOVERY")
print("=" * 60)
print(f"   Searching for {len(ALL_BUSINESSES)} businesses in {CITY}...\n")

# Initialize discovery engine
engine = DiscoveryEngine(debug=True)

# Output path for discovered locations
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
discovery_output = project_root / "data" / "03_processed" / "discovery" / f"demo_discovery_{timestamp}.csv"
discovery_output.parent.mkdir(parents=True, exist_ok=True)

# Run discovery
discovered_df = engine.discover_branches(
    businesses=ALL_BUSINESSES,
    cities=[CITY],
    business_type=BUSINESS_TYPE,
    output_path=discovery_output,
)

print(f"\nDISCOVERY COMPLETE")
print("=" * 60)
print(f"   Total locations found: {len(discovered_df)}")
print(f"   Output saved: {discovery_output.name}")

# Show results by business
if 'business' in discovered_df.columns:
    print(f"\n   Locations per business:")
    for biz in ALL_BUSINESSES:
        count = len(discovered_df[discovered_df['business'] == biz])
        print(f"      - {biz}: {count} locations")


STEP 1: DISCOVERY
   Searching for 3 businesses in Casablanca...



Resolving place_ids: 100%|██████████| 66/66 [00:00<00:00, 20618.51it/s]


DISCOVERY COMPLETE
   Total locations found: 66
   Output saved: demo_discovery_20251126_154328.csv

   Locations per business:
      - RMA Assurance: 22 locations
      - Wafa Assurance: 24 locations
      - Axa Assurance Maroc: 20 locations


In [13]:
# Preview discovered locations
print("DISCOVERED LOCATIONS (Preview)")
print("=" * 60)

# Select display columns
display_cols = ['business', 'name', 'address', 'city']
display_cols = [c for c in display_cols if c in discovered_df.columns]

# Add place_id column
id_col = 'canonical_place_id' if 'canonical_place_id' in discovered_df.columns else 'place_id'
if id_col in discovered_df.columns:
    display_cols.append(id_col)

display(discovered_df[display_cols].head(15))


DISCOVERED LOCATIONS (Preview)


,business,name,address,city,canonical_place_id
0,RMA Assurance,RMA Assurance_Siège Social Casablanca.,"Av. des FAR, Casablanca, Maroc",Casablanca,ChIJZW_wW33Npw0RH8nG1MACfsQ
1,RMA Assurance,RMA Assurance,"32 Boulevard Bir Anzarane, Casablanca, Maroc",Casablanca,ChIJl54keBjTpw0R2yatLm5l0CA
2,RMA Assurance,RMA Assurance,"Hay El Hanaa, 25, Bd Sidi Abderrahman, Casabla...",Casablanca,ChIJQxgHxz7Tpw0ReYhop_EIA-w
3,RMA Assurance,RMA INTEGRITE ASSURANCES,"12 Résidence KOUTOUBIA Nassim, Casablanca 2019...",Casablanca,ChIJ_dppWPsspg0ROW9ZXdgyT6U
4,RMA Assurance,RMA Assurance,"153 Bd Moulay Idriss I, Casablanca 20250, Maroc",Casablanca,ChIJzZQFQq_Spw0RrJPQa93duKk
5,RMA Assurance,RMA Assurance,"G9J2+XVP, Casablanca, Maroc",Casablanca,ChIJ4d0M3xstpg0RVJMtnC9sd9g
6,RMA Assurance,RMA Assurance,"83, Av. de l'Arm?e Royale -, Casablanca 20000,...",Casablanca,ChIJH7TxrILSpw0RTtZTKkSiMe8
7,RMA Assurance,RMA Assurance,"GCP7+CCC, Casablanca 20500, Maroc",Casablanca,ChIJL2FRGcUzpg0RAlEJw8ILozI
8,RMA Assurance,Centre Rma Service,"15 Rue Omar Slaoui, Casablanca 20670, Maroc",Casablanca,ChIJ2aHpHj_Tpw0RoLfBMzrAXE8
9,RMA Assurance,RMA Assurance,"91 Bd Moulay Hassan I, Casablanca 20100, Maroc",Casablanca,ChIJ_VSFx5rSpw0RT059lIST6fQ


In [14]:
# Limit locations per business for demo
print(f"Limiting to {MAX_LOCATIONS_PER_BUSINESS} locations per business for demo speed...\n")

limited_dfs = []
for biz in ALL_BUSINESSES:
    biz_df = discovered_df[discovered_df['business'] == biz].head(MAX_LOCATIONS_PER_BUSINESS)
    limited_dfs.append(biz_df)
    print(f"   {biz}: {len(biz_df)} locations")

demo_locations_df = pd.concat(limited_dfs, ignore_index=True)
print(f"\n   Total locations for demo: {len(demo_locations_df)}")

# Save limited locations for collection
demo_locations_path = project_root / "data" / "02_interim" / "demo" / f"demo_locations_{timestamp}.csv"
demo_locations_path.parent.mkdir(parents=True, exist_ok=True)

# Prepare for collection (rename columns to expected format)
# Include lat/lng and other metadata for downstream processing
id_col = 'canonical_place_id' if 'canonical_place_id' in demo_locations_df.columns else 'place_id'
collection_df = pd.DataFrame({
    '_place_id': demo_locations_df[id_col] if id_col in demo_locations_df.columns else demo_locations_df.get('data_id'),
    '_business': demo_locations_df['business'],
    '_city': demo_locations_df.get('city', CITY),
    'title': demo_locations_df.get('name', ''),
    'address': demo_locations_df.get('address', ''),
    'lat': demo_locations_df.get('lat'),
    'lng': demo_locations_df.get('lng'),
    'place_rating': demo_locations_df.get('rating'),
    'reviews_count': demo_locations_df.get('reviews_count'),
}).dropna(subset=['_place_id'])

collection_df.to_csv(demo_locations_path, index=False)
print(f"   Saved to: {demo_locations_path.name}")
print(f"   Columns: {list(collection_df.columns)}")


Limiting to 5 locations per business for demo speed...

   RMA Assurance: 5 locations
   Wafa Assurance: 5 locations
   Axa Assurance Maroc: 5 locations

   Total locations for demo: 15
   Saved to: demo_locations_20251126_154328.csv
   Columns: ['_place_id', '_business', '_city', 'title', 'address', 'lat', 'lng', 'place_rating', 'reviews_count']


---

## Step 2: Collection

Collect Google Maps reviews for discovered locations.

**What happens:**
- Uses SerpAPI to fetch reviews from Google Maps
- Collects review text, rating, date, author
- Limited to MAX_REVIEWS_PER_LOCATION per location


In [15]:
# ==============================================================================
# STEP 2: COLLECTION — Collect Reviews
# ==============================================================================

print("STEP 2: COLLECTION")
print("=" * 60)
print(f"   Collecting reviews from {len(collection_df)} locations...")
print(f"   (Will limit to {MAX_REVIEWS_PER_LOCATION} reviews per location after collection)\n")

# Initialize collector
collector = ReviewCollector(debug=True)

# Output path
collection_output = project_root / "data" / "02_interim" / "demo" / f"demo_reviews_{timestamp}.csv"

# Collect reviews (no limit parameter - we'll truncate afterwards)
stats = collector.collect_reviews(
    input_file=demo_locations_path,
    output_mode="csv",
    output_path=collection_output,
)

print(f"\nCOLLECTION COMPLETE")
print("=" * 60)
for key, value in stats.items():
    print(f"   {key}: {value}")


STEP 2: COLLECTION
   (Will limit to 5 reviews per location after collection)



API error on page 19 for ChIJQxgHxz7Tpw0ReYhop_EIA-w: Google hasn't returned any results for this query.
API error on page 29 for ChIJYZPt0sMSpw0Rp6MTUn0WEa8: Google hasn't returned any results for this query.
API error on page 37 for ChIJmdau85fSpw0Rg0X4ArIPLPs: Google hasn't returned any results for this query.


COLLECTION COMPLETE
   status: success
   total_places: 15
   agencies_processed: 15
   total_reviews: 1386
   output_mode: csv


In [16]:
# Load collected reviews
reviews_df_full = pd.read_csv(collection_output)

print("COLLECTED REVIEWS")
print("=" * 60)
print(f"   Total reviews collected: {len(reviews_df_full)}")

# Limit reviews per location for demo speed
place_id_col = '_place_id' if '_place_id' in reviews_df_full.columns else 'place_id'
if place_id_col in reviews_df_full.columns:
    print(f"\n   Limiting to {MAX_REVIEWS_PER_LOCATION} reviews per location...")
    reviews_df = reviews_df_full.groupby(place_id_col).head(MAX_REVIEWS_PER_LOCATION).reset_index(drop=True)
    print(f"   Reviews after limiting: {len(reviews_df)}")
else:
    print(f"   Warning: Could not find place_id column, using all reviews")
    reviews_df = reviews_df_full.copy()

# Show distribution by business
business_col = '_business' if '_business' in reviews_df.columns else '_bank'
if business_col in reviews_df.columns:
    print(f"\n   Reviews per business:")
    for biz, count in reviews_df[business_col].value_counts().items():
        print(f"      - {biz}: {count} reviews")

# Show rating distribution
if 'rating' in reviews_df.columns:
    print(f"\n   Rating distribution:")
    for rating in sorted(reviews_df['rating'].dropna().unique()):
        count = (reviews_df['rating'] == rating).sum()
        print(f"      {int(rating)} stars: {count}")

# Preview sample reviews
print(f"\n   Sample reviews:")
preview_cols = [c for c in [business_col, 'rating', 'text'] if c in reviews_df.columns]
display(reviews_df[preview_cols].head(5))


COLLECTED REVIEWS
   Total reviews collected: 1386

   Limiting to 5 reviews per location...
   Reviews after limiting: 65

   Reviews per business:
      - RMA Assurance: 25 reviews
      - Wafa Assurance: 25 reviews
      - Axa Assurance Maroc: 15 reviews

   Rating distribution:
      1 stars: 31
      3 stars: 1
      4 stars: 2
      5 stars: 31

   Sample reviews:


,_business,rating,text
0,RMA Assurance,1.0,Je souhaite exprimer mon insatisfaction concer...
1,RMA Assurance,1.0,"Service très nul, personnel malhonnête et pas ..."
2,RMA Assurance,1.0,Impossible de joindre le service réclamation m...
3,RMA Assurance,1.0,Actuellement ma voiture est en réparation suit...
4,RMA Assurance,1.0,"Je suis assuré en tous risques depuis un an, e..."


---

## Step 3: Transform

Normalize and enrich review data before classification.

**What happens:**
- Parse French relative dates ("il y a 2 mois" → datetime)
- Normalize city names
- Add geographic regions
- Clean text fields


In [17]:
# ==============================================================================
# STEP 3: TRANSFORM — Normalize Reviews
# ==============================================================================

print("STEP 3: TRANSFORM")
print("=" * 60)
print(f"   Normalizing {len(reviews_df)} reviews...\n")

# Apply normalization
normalized_df = normalize_reviews_df(reviews_df.copy())

# Add regions if lat/lng columns exist and regions.geojson is available
regions_file = config.REGIONS_FILE if hasattr(config, 'REGIONS_FILE') else project_root / "data" / "00_config" / "cities" / "regions.geojson"
has_coords = any(col in normalized_df.columns for col in ['lat', 'latitude']) and \
             any(col in normalized_df.columns for col in ['lng', 'longitude'])

if regions_file.exists() and has_coords:
    print("   Adding geographic regions...")
    try:
        normalized_df = add_region(normalized_df, regions_path=regions_file)
        regions_added = normalized_df['region'].notna().sum() if 'region' in normalized_df.columns else 0
        print(f"   Regions assigned: {regions_added}/{len(normalized_df)}")
    except Exception as e:
        print(f"   Warning: Could not add regions: {e}")
elif not has_coords:
    print(f"   Note: No lat/lng columns found, skipping region assignment")
    print(f"   Available columns: {list(normalized_df.columns)}")
else:
    print(f"   Note: regions.geojson not found, skipping region assignment")

print(f"\nTRANSFORM COMPLETE")
print("=" * 60)

# Show new columns
new_cols = set(normalized_df.columns) - set(reviews_df.columns)
if new_cols:
    print(f"   New columns added: {sorted(new_cols)}")

# Show sample
sample_cols = [c for c in ['created_at', 'rating', 'city_normalized', 'region'] if c in normalized_df.columns]
if sample_cols:
    print(f"\n   Sample transformed data:")
    display(normalized_df[sample_cols].head(5))

# Save transformed data
transform_output = project_root / "data" / "02_interim" / "demo" / f"demo_reviews_transformed_{timestamp}.csv"
normalized_df.to_csv(transform_output, index=False)
print(f"\n   Saved to: {transform_output.name}")


STEP 3: TRANSFORM
   Normalizing 65 reviews...

   Note: No lat/lng columns found, skipping region assignment
   Available columns: ['_place_id', '_city', '_business', 'title', 'date', 'rating', 'text', 'author', 'created_at', 'month']

TRANSFORM COMPLETE
   New columns added: ['created_at', 'month']

   Sample transformed data:


,created_at,rating
0,2025-10-27 14:48:08.656752,1
1,2025-08-28 14:48:08.656765,1
2,2025-09-27 14:48:08.656769,1
3,2025-06-29 14:48:08.656772,1
4,2025-04-30 14:48:08.656775,1



   Saved to: demo_reviews_transformed_20251126_154328.csv


---

## Step 4: Classification

Classify reviews using OpenAI GPT-4 into 17 predefined categories.

**What happens:**
- Each review is analyzed for sentiment (Positif/Négatif/Neutre)
- Categories are assigned with confidence scores
- Few-shot learning improves accuracy
- Results are saved in wide format (binary columns for each category)


In [18]:
# ==============================================================================
# STEP 4: CLASSIFICATION — Classify Reviews with OpenAI
# ==============================================================================

print("STEP 4: CLASSIFICATION")
print("=" * 60)
print(f"   Classifying {len(normalized_df)} reviews with OpenAI GPT-4...")
print(f"   Categories: {len(config.CATEGORIES)}")
print(f"   Confidence threshold: {config.CONF_THRESHOLD}\n")

# Initialize classifier
classifier = ReviewClassifier(debug=True)

# Classify reviews
classified_df = classifier.classify_batch(normalized_df.copy())

# Convert to wide format (binary category columns)
wide_df = classifier.convert_to_wide_format(classified_df)

print(f"\nCLASSIFICATION COMPLETE")
print("=" * 60)
print(f"   Reviews classified: {len(wide_df)}")

# Sentiment distribution
if 'sentiment' in wide_df.columns:
    print(f"\n   Sentiment distribution:")
    for sent, count in wide_df['sentiment'].value_counts().items():
        pct = count / len(wide_df) * 100
        print(f"      - {sent}: {count} ({pct:.1f}%)")

# Save classified data
classification_output = project_root / "data" / "03_processed" / "demo" / f"demo_reviews_classified_{timestamp}.csv"
classification_output.parent.mkdir(parents=True, exist_ok=True)
wide_df.to_csv(classification_output, index=False)
print(f"\n   Saved to: {classification_output.name}")


STEP 4: CLASSIFICATION
   Classifying 65 reviews with OpenAI GPT-4...
   Categories: 17
   Confidence threshold: 0.55



Classifying: 100%|██████████| 65/65 [02:44<00:00,  2.53s/it]


CLASSIFICATION COMPLETE
   Reviews classified: 65

   Sentiment distribution:
      - Positif: 34 (52.3%)
      - Négatif: 27 (41.5%)
      - Neutre: 2 (3.1%)

   Saved to: demo_reviews_classified_20251126_154328.csv


In [19]:
# Preview classification results
print("CLASSIFICATION RESULTS (Preview)")
print("=" * 60)

# Detect columns
text_col = 'text' if 'text' in wide_df.columns else 'review_text'
business_col = '_business' if '_business' in wide_df.columns else '_bank'

# Show sample with sentiment and rationale
preview_cols = [c for c in [business_col, 'rating', 'sentiment', 'rationale'] if c in wide_df.columns]
display(wide_df[preview_cols].head(10))

# Category columns
cat_cols = [c for c in wide_df.columns if c in config.CATEGORIES]
print(f"\n   Category columns created: {len(cat_cols)}")

# Top categories
if cat_cols:
    cat_sums = wide_df[cat_cols].sum().sort_values(ascending=False)
    print(f"\n   Top 5 categories mentioned:")
    for cat, count in cat_sums.head(5).items():
        short_name = cat[:50] + "..." if len(cat) > 50 else cat
        print(f"      - {short_name}: {int(count)} reviews")


CLASSIFICATION RESULTS (Preview)


,_business,rating,sentiment,rationale
0,RMA Assurance,1,Négatif,"Réclamation non traitée malgré appels répétés,..."
1,RMA Assurance,1,Négatif,"Critique du personnel peu professionnel, servi..."
2,RMA Assurance,1,Négatif,Service client injoignable et problème techniq...
3,RMA Assurance,1,Négatif,Client se plaint d'un service client inexistan...
4,RMA Assurance,1,Négatif,Critique sur la transparence et la couverture ...
5,RMA Assurance,5,Positif,Avis positif sans détails spécifiques.
6,RMA Assurance,5,Positif,"Remerciement pour service professionnel, menti..."
7,RMA Assurance,5,Positif,"Le client souligne l'écoute, le professionnali..."
8,RMA Assurance,5,Positif,Avis positif sans détail spécifique.
9,RMA Assurance,5,Positif,"Client souligne écoute et suivi, recommandatio..."



   Category columns created: 17

   Top 5 categories mentioned:
      - Conseil personnalisé et professionnalisme des équi...: 20 reviews
      - Service client réactif et à l'écoute (problèmes ré...: 18 reviews
      - Accueil chaleureux et personnel attentionné (expér...: 16 reviews
      - Manque de considération ou attitude peu profession...: 13 reviews
      - Réclamations ignorées ou mal suivies (absence de r...: 11 reviews


---

## Step 5: EDA & Visualizations

Create visualizations to analyze the classified reviews.

**Charts included:**
1. Rating Distribution (bar chart)
2. Sentiment Breakdown (pie chart)
3. Average Rating by Business (bar chart)
4. Top Categories (bar chart)
5. Category Heatmap (BCG-style)

**Note:** If you restarted the kernel, run the cell below to reload the classified data from disk.


In [2]:
# ==============================================================================
# OPTIONAL: Reload classified data from disk (if kernel was restarted)
# ==============================================================================
# Uncomment and run this cell if you restarted the kernel and want to skip
# the discovery, collection, transform, and classification steps.

from pathlib import Path
import pandas as pd
from review_analyzer import config
 
# Set the timestamp of the run you want to load
SAVED_TIMESTAMP = "20251126_154328"  # <-- Change this to your saved timestamp

project_root = Path().resolve().parent
classification_output = project_root / "data" / "03_processed" / "demo" / f"demo_reviews_classified_{SAVED_TIMESTAMP}.csv"
# 
if classification_output.exists():
    wide_df = pd.read_csv(classification_output)
    print(f"Loaded {len(wide_df)} classified reviews from: {classification_output.name}")
    print(f"Columns: {list(wide_df.columns)}")
else:
    print(f"File not found: {classification_output}")
    # List available files
    demo_dir = project_root / "data" / "03_processed" / "demo"
    if demo_dir.exists():
        files = list(demo_dir.glob("demo_reviews_classified_*.csv"))
    print(f"Available files: {[f.name for f in files]}")


Loaded 65 classified reviews from: demo_reviews_classified_20251126_154328.csv
Columns: ['_place_id', '_city', '_business', 'title', 'date', 'rating', 'text', 'author', 'created_at', 'month', 'sentiment', 'categories_json', 'language', 'rationale', "Accueil chaleureux et personnel attentionné (expérience humaine positive, sentiment d'être bien accueilli)", "Service client réactif et à l'écoute (problèmes résolus rapidement, vraie disponibilité)", 'Conseil personnalisé et professionnalisme des équipes (expertise perçue, accompagnement individualisé)', "Efficacité et rapidité de traitement (fluidité, peu d'attente, processus clairs)", "Accessibilité et proximité des services (agences, guichets, présence locale, simplicité d'accès)", "Satisfaction sans détails spécifiques (le client indique que le service ou l'agence est bien sans explication )", 'Expérience digitale et services en ligne pratiques (application fluide, opérations faciles à distance )', "Attente interminable et lenteur en a

In [3]:
# ==============================================================================
# EDA SETUP — Theme and Helper Functions
# ==============================================================================

# Fix for nbformat issue: reload modules after pip install (no kernel restart needed)
import importlib
try:
    import nbformat
    importlib.reload(nbformat)
    print(f"   nbformat version: {nbformat.__version__}")
except ImportError:
    print("   Warning: nbformat not found, run: pip install nbformat>=4.2.0")

# Reload plotly.io to pick up the reloaded nbformat
import plotly.io as pio
importlib.reload(pio)
print("   Plotly modules reloaded")

# BCG Theme Colors
BCG_GREEN = "#0B6E4F"
BCG_DARK_GREEN = "#003D32"
BCG_LIGHT_GREEN = "#A7C957"
BCG_TEAL = "#1F7A8C"
BCG_GRAY = "#5C7C89"
BCG_RED = "#D62728"
BCG_ORANGE = "#FF7F0E"
BCG_YELLOW = "#FFD700"

RATING_COLORS = {
    1: "#D62728",  # Red
    2: "#FF7F0E",  # Orange
    3: "#FFD700",  # Yellow
    4: "#A7C957",  # Light green
    5: "#0B6E4F",  # BCG green
}

SENTIMENT_COLORS = {
    "Positif": BCG_GREEN,
    "Négatif": BCG_RED,
    "Neutre": BCG_GRAY,
}

# Short category names for display
def shorten_category(cat: str, max_len: int = 35) -> str:
    """Shorten category name for display."""
    short = cat.split("(")[0].strip()
    if len(short) > max_len:
        short = short[:max_len-3] + "..."
    return short

print("EDA theme loaded")


   nbformat version: 5.10.4
   Plotly modules reloaded
EDA theme loaded


In [4]:
# ==============================================================================
# VISUALIZATION 1: Rating Distribution
# ==============================================================================

print("VISUALIZATION 1: Rating Distribution")
print("=" * 60)

if 'rating' in wide_df.columns:
    rating_counts = wide_df['rating'].value_counts().sort_index()
    
    fig = go.Figure(data=[
        go.Bar(
            x=[f"{int(r)} stars" for r in rating_counts.index],
            y=rating_counts.values,
            marker_color=[RATING_COLORS.get(int(r), BCG_GRAY) for r in rating_counts.index],
            text=rating_counts.values,
            textposition='outside',
        )
    ])
    
    fig.update_layout(
        title=f"<b>Rating Distribution</b><br><sup>{len(wide_df)} reviews analyzed</sup>",
        xaxis_title="Rating",
        yaxis_title="Number of Reviews",
        height=400,
        showlegend=False,
    )
    
    fig.show()
else:
    print("   Rating column not found")


VISUALIZATION 1: Rating Distribution


In [5]:
# ==============================================================================
# VISUALIZATION 2: Sentiment Breakdown
# ==============================================================================

print("VISUALIZATION 2: Sentiment Breakdown")
print("=" * 60)

if 'sentiment' in wide_df.columns:
    sentiment_counts = wide_df['sentiment'].value_counts()
    
    colors = [SENTIMENT_COLORS.get(s, BCG_GRAY) for s in sentiment_counts.index]
    
    fig = go.Figure(data=[
        go.Pie(
            labels=sentiment_counts.index,
            values=sentiment_counts.values,
            marker_colors=colors,
            textinfo='label+percent',
            textfont_size=14,
            hole=0.4,
        )
    ])
    
    fig.update_layout(
        title=f"<b>Sentiment Distribution</b><br><sup>{len(wide_df)} reviews</sup>",
        height=450,
        showlegend=True,
    )
    
    fig.show()
else:
    print("   Sentiment column not found")


VISUALIZATION 2: Sentiment Breakdown


In [6]:
# ==============================================================================
# VISUALIZATION 3: Average Rating by Business
# ==============================================================================

print("VISUALIZATION 3: Average Rating by Business")
print("=" * 60)

business_col = '_business' if '_business' in wide_df.columns else '_bank'

if business_col in wide_df.columns and 'rating' in wide_df.columns:
    avg_ratings = wide_df.groupby(business_col).agg(
        avg_rating=('rating', 'mean'),
        n_reviews=('rating', 'size')
    ).reset_index().sort_values('avg_rating', ascending=True)
    
    fig = go.Figure(data=[
        go.Bar(
            x=avg_ratings['avg_rating'],
            y=avg_ratings[business_col],
            orientation='h',
            marker_color=BCG_GREEN,
            text=[f"{r:.2f} (n={n})" for r, n in zip(avg_ratings['avg_rating'], avg_ratings['n_reviews'])],
            textposition='outside',
        )
    ])
    
    fig.update_layout(
        title=f"<b>Average Rating by Business</b><br><sup>Comparison across {len(avg_ratings)} businesses</sup>",
        xaxis_title="Average Rating (1-5)",
        yaxis_title="",
        height=max(300, len(avg_ratings) * 60),
        xaxis=dict(range=[0, 5.5]),
    )
    
    fig.show()
else:
    print("   Required columns not found")


VISUALIZATION 3: Average Rating by Business


In [7]:
# ==============================================================================
# VISUALIZATION 4: Top Categories (Bar Chart)
# ==============================================================================

print("VISUALIZATION 4: Top Categories Mentioned")
print("=" * 60)

# Get category columns
cat_cols = [c for c in wide_df.columns if c in config.CATEGORIES]

if cat_cols:
    # Sum occurrences
    cat_sums = wide_df[cat_cols].sum().sort_values(ascending=True)
    
    # Take top 10
    top_cats = cat_sums.tail(10)
    
    # Shorten names for display
    short_names = [shorten_category(c) for c in top_cats.index]
    
    # Determine colors based on category type
    colors = []
    for cat in top_cats.index:
        cat_lower = cat.lower()
        if any(neg in cat_lower for neg in ['attente', 'lenteur', 'injoignable', 'réclamation', 'incident', 'frais', 'insatisfaction', 'manque']):
            colors.append(BCG_RED)
        elif 'autre' in cat_lower:
            colors.append(BCG_GRAY)
        else:
            colors.append(BCG_GREEN)
    
    fig = go.Figure(data=[
        go.Bar(
            x=top_cats.values,
            y=short_names,
            orientation='h',
            marker_color=colors,
            text=top_cats.values.astype(int),
            textposition='outside',
        )
    ])
    
    fig.update_layout(
        title=f"<b>Top 10 Categories Mentioned</b><br><sup>Green = Positive, Red = Negative</sup>",
        xaxis_title="Number of Reviews",
        yaxis_title="",
        height=500,
    )
    
    fig.show()
else:
    print("   No category columns found")


VISUALIZATION 4: Top Categories Mentioned


In [8]:
# ==============================================================================
# VISUALIZATION 5: Category Heatmap (BCG Style)
# ==============================================================================

print("VISUALIZATION 5: Category Heatmap by Business")
print("=" * 60)

business_col = '_business' if '_business' in wide_df.columns else '_bank'
cat_cols = [c for c in wide_df.columns if c in config.CATEGORIES]

if cat_cols and business_col in wide_df.columns:
    # Build matrix: rows = categories, columns = businesses
    businesses = sorted(wide_df[business_col].dropna().unique())
    
    # Calculate % for each business-category combination
    matrix_data = []
    for cat in cat_cols:
        row = {'Category': shorten_category(cat)}
        for biz in businesses:
            sub = wide_df[wide_df[business_col] == biz]
            if len(sub) > 0:
                pct = sub[cat].sum() / len(sub) * 100
            else:
                pct = 0
            row[biz] = round(pct, 1)
        matrix_data.append(row)
    
    matrix_df = pd.DataFrame(matrix_data).set_index('Category')
    
    # Filter to categories with at least some data
    matrix_df = matrix_df[matrix_df.sum(axis=1) > 0]
    
    if len(matrix_df) > 0:
        # Create heatmap
        fig = go.Figure(data=go.Heatmap(
            z=matrix_df.values,
            x=matrix_df.columns.tolist(),
            y=matrix_df.index.tolist(),
            colorscale=[[0, '#F7FCF5'], [0.5, '#74C476'], [1, '#00441B']],
            text=[[f"{v:.1f}%" for v in row] for row in matrix_df.values],
            texttemplate="%{text}",
            textfont=dict(size=10),
            hovertemplate="Category: %{y}<br>Business: %{x}<br>Percentage: %{z:.1f}%<extra></extra>",
            colorbar=dict(title="%"),
        ))
        
        fig.update_layout(
            title=f"<b>Category Distribution by Business</b><br><sup>Percentage of reviews mentioning each category</sup>",
            xaxis_title="Business",
            yaxis_title="",
            height=max(500, len(matrix_df) * 35 + 150),
            xaxis=dict(tickangle=-45),
            yaxis=dict(autorange='reversed'),
        )
        
        fig.show()
    else:
        print("   No category data to display")
else:
    print("   Required columns not found")


VISUALIZATION 5: Category Heatmap by Business


In [9]:
# ==============================================================================
# VISUALIZATION 6: Positive vs Negative Categories Comparison
# ==============================================================================

print("VISUALIZATION 6: Positive vs Negative Factors")
print("=" * 60)

cat_cols = [c for c in wide_df.columns if c in config.CATEGORIES]

if cat_cols:
    # Separate positive and negative categories
    pos_cats = []
    neg_cats = []
    
    for cat in cat_cols:
        cat_lower = cat.lower()
        if any(neg in cat_lower for neg in ['attente', 'lenteur', 'injoignable', 'réclamation', 'incident', 'frais', 'insatisfaction', 'manque']):
            neg_cats.append(cat)
        elif 'autre' not in cat_lower and 'hors-sujet' not in cat_lower:
            pos_cats.append(cat)
    
    if pos_cats and neg_cats:
        # Calculate totals
        pos_total = wide_df[pos_cats].sum().sum()
        neg_total = wide_df[neg_cats].sum().sum()
        
        fig = go.Figure(data=[
            go.Bar(
                x=['Positive Factors', 'Negative Factors'],
                y=[pos_total, neg_total],
                marker_color=[BCG_GREEN, BCG_RED],
                text=[int(pos_total), int(neg_total)],
                textposition='outside',
            )
        ])
        
        fig.update_layout(
            title=f"<b>Positive vs Negative Factors</b><br><sup>Total category mentions across all reviews</sup>",
            yaxis_title="Number of Mentions",
            height=400,
            showlegend=False,
        )
        
        fig.show()
        
        # Print summary
        print(f"\n   Summary:")
        print(f"      Positive factors: {int(pos_total)} mentions ({len(pos_cats)} categories)")
        print(f"      Negative factors: {int(neg_total)} mentions ({len(neg_cats)} categories)")
        ratio = pos_total / neg_total if neg_total > 0 else float('inf')
        print(f"      Ratio (Pos/Neg): {ratio:.2f}")
    else:
        print("   Could not separate positive/negative categories")
else:
    print("   No category columns found")


VISUALIZATION 6: Positive vs Negative Factors



   Summary:
      Positive factors: 62 mentions (6 categories)
      Negative factors: 46 mentions (8 categories)
      Ratio (Pos/Neg): 1.35


---

## Demo Complete

### Summary

This demo showed the complete pipeline:

1. **Discovery** — Found Google Maps locations for businesses in the target city
2. **Collection** — Collected reviews from discovered locations
3. **Transform** — Normalized dates, cleaned text, added regions
4. **Classification** — Classified reviews into 17 categories using OpenAI
5. **EDA** — Visualized results with ratings, sentiment, and category heatmaps

### Output Files

All demo data has been saved to:
- `data/02_interim/demo/` — Intermediate files
- `data/03_processed/demo/` — Final classified results

### Next Steps

For production use:
- Remove the demo limits (MAX_LOCATIONS_PER_BUSINESS, MAX_REVIEWS_PER_LOCATION)
- Run for all cities and all competitors
- Export results to Excel or connect to a dashboard


In [ ]:
# ==============================================================================
# DEMO SUMMARY
# ==============================================================================

print("\n" + "=" * 60)
print("DEMO COMPLETE!")
print("=" * 60)

print(f"\nConfiguration:")
print(f"   Business Type: {BUSINESS_TYPE}")
print(f"   Target Business: {TARGET_BUSINESS}")
print(f"   Competitors: {COMPETITORS}")
print(f"   City: {CITY}")

print(f"\nResults:")
print(f"   Locations discovered: {len(demo_locations_df)}")
print(f"   Reviews collected: {len(reviews_df)}")
print(f"   Reviews classified: {len(wide_df)}")

if 'sentiment' in wide_df.columns:
    pos_pct = (wide_df['sentiment'] == 'Positif').mean() * 100
    neg_pct = (wide_df['sentiment'] == 'Négatif').mean() * 100
    print(f"\n   Sentiment:")
    print(f"      Positive: {pos_pct:.1f}%")
    print(f"      Negative: {neg_pct:.1f}%")

if 'rating' in wide_df.columns:
    avg_rating = wide_df['rating'].mean()
    print(f"\n   Average Rating: {avg_rating:.2f} / 5")

print(f"\nOutput Files:")
print(f"   {classification_output}")

print(f"\nReady for presentation!")
